Lab 4: https://kbodwin.github.io/GSB-544-private/Lab4.html


## Scrape the Location Counts

1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:
- Starbucks
- Dunkin’ Donuts

2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

In [1]:
import pandas as pd
import requests as re
from bs4 import BeautifulSoup

In [2]:
response_st = re.get('https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state')

In [3]:
soup_st = BeautifulSoup(response_st.content, 'html.parser')

In [4]:
table_st = soup_st.find_all('table',
                attrs={
                    'class': 'wpr-table'
                })

In [5]:
len(table_st)

1

1. Creating function to scrape data and create a dataframe

In [6]:
# future function (to be called within a main function) in Step 4. Automation
def ws_to_df(url):
    # scraping the data with BeautifulSoup
    response = re.get(url)
    soup_table = BeautifulSoup(response.content, 'html.parser')

    # Parsing all rows from the table
    rows = soup_table.find_all('tr')

    # Extracting headers (if present)
    headers = [header.text.strip() for header in rows[0].find_all('th')]

    # Initializing a list for the table
    table_data = []

    # Looping through rows to extract data
    for row in rows[1:]: # skipping header extracted in prior step
        state = row.find('a').text
        values = [state] + [td.text.strip().replace(',','') for td in row.find_all('td')] ## assisted by ChatGPT
        table_data.append(values)
    
    # Creating dataframe
    df = pd.DataFrame(table_data, columns=headers)
    return(df)

# note: Originally used an adjusted code from the PA assignment, however, while trying to fix a bug, managed to get a more elegant code suggestion from ChatGPT.

NOTE: For the purpose of analyzing 'current state of the market', it makes sense to focus only on all state data (without update to 2024, to avoid infalting the numbers - eg. opened +37 stores in 2024 in CA (reflected in the original data), but closed -5 stores in MN (not info on 2024 in table). Therefore, I'm going to use only original data for 2023 for further analysis.

In [7]:
sb_df = ws_to_df('https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state')

In [8]:
dd_df = ws_to_df('https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state')

In [9]:
# Starbucks data
# sb_df

In [10]:
# Dunkin Donuts data
# dd_df

Due to partial data for Starbucks in 2024, and not data for Dunkin in 2021, it makes sense to limit the analysis only to most recent full data for 2023 for both brands.

In [11]:
sb_loc_2023 = sb_df[['State','Starbucks Stores 2023']]
# sb_loc_2023

In [12]:
dd_loc_2023 = dd_df[['State','Dunkin Locations 2023']]
# dd_loc_2023

### ! fact check intervention !
one thing easy thing to catch about Dunkin data is the change in number of locations between state of Washington and District of Columbia. After doublechecking on their website (https://locations.dunkindonuts.com/en) it looks like there are 19 stores in Washington D.C. but not in the state of Washington. Therefore, manually correcting this in the 'tidy' dataset for Dunkin.

In [13]:
# reassigning the values in corresponding cells
dd_loc_2023.iloc[36,1] = 19
dd_loc_2023.iloc[50,1] = 0

<positron-console-cell-13>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<positron-console-cell-13>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
# dd_loc_2023

In [32]:
# merging data for two chains by state
merged_chains_df = pd.merge(sb_loc_2023, dd_loc_2023, on='State')

In [33]:
# renaming one of the columns to match
merged_chains_df = merged_chains_df.rename(columns={'Dunkin Locations 2023':'Dunkin Stores 2023'})

In [34]:
merged_chains_df.head()

,State,Starbucks Stores 2023,Dunkin Stores 2023
0,California,3080,134
1,Texas,1346,196
2,Florida,844,883
3,Washington,741,0
4,New York,692,1414


## 2. Supplemental data

In [17]:
response_wiki = re.get('https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population')

In [18]:
soup_wiki = BeautifulSoup(response_wiki.content, 'html.parser')

In [19]:
# Parsing table and creating dataframe. Note: assisted by ChatGPT
states = []
populations = []
regions = []

# Predefined vectors for each region
Northeast = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 
             'Vermont', 'New Jersey', 'New York', 'Pennsylvania']
Midwest = ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin', 
           'Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 
           'North Dakota', 'South Dakota']
South = ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 
         'South Carolina', 'Virginia', 'District of Columbia', 'West Virginia', 
         'Alabama', 'Kentucky', 'Mississippi', 'Tennessee', 'Arkansas', 
         'Louisiana', 'Oklahoma', 'Texas']
West = ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 
        'New Mexico', 'Utah', 'Wyoming', 'Alaska', 
        'California', 'Hawaii', 'Oregon', 'Washington']

# Function to assign region based on State to further use in the loop for creating a dataframe
def get_region(state):
    if state in Northeast:
        return "Northeast"
    elif state in Midwest:
        return "Midwest"
    elif state in South:
        return "South"
    elif state in West:
        return "West"
    else:
        return "Unknown"

# Looping through rows and extracting required columns
for row in soup_wiki.find_all("tr")[1:]:  # Skip header row
    cols = row.find_all("td")
    if len(cols) >= 4:
        state = cols[2].get_text(strip=True)
        population = cols[3].get_text(strip=True).replace(",", "")
        region = get_region(state)
        states.append(state)
        populations.append(int(population))
        regions.append(region)



# Creating DataFrame
df_pop = pd.DataFrame({"State": states, "Population": populations, "Region": regions})


In [20]:
df_pop.head()

,State,Population,Region
0,California,39538223,West
1,Texas,30145505,South
2,Florida,21538187,South
3,New York,20201249,Northeast
4,Pennsylvania,13002700,Northeast


merging datasets with chain locations and population

In [35]:
merged_df = pd.merge(merged_chains_df, df_pop, on='State')

In [ ]:
# merged_df.info()

In [36]:
# chaning store location data to numeric
merged_df['Starbucks Stores 2023'] = pd.to_numeric(merged_df['Starbucks Stores 2023'], errors='coerce')
merged_df['Dunkin Stores 2023'] = pd.to_numeric(merged_df['Dunkin Stores 2023'], errors='coerce')

## Adding financial information for companies:

## NOTE: 
Considering there is barely any open, trustworthy and verifiable financial data available for Dunkin in 2023, after they were acquired by Arby's in 2020, this part of the analysis is rather limited, in order to keep it 'apples to apples' with Starbucks. Hence, the only reasonable 'metric' (from the same source: Zippia) that I found for for both business are 2023 revenues:

    Dunkin Donuts: 1.4 (in billion USD) (source: https://www.zippia.com/dunkin-donuts-careers-554008/revenue/)
    
    Startbucks: 36.0 (in billion USD) (source: https://www.zippia.com/starbucks-careers-10803/revenue/)

Other metrics from this source don't look accuarte, like number of employees etc.

Based on this metric, I can estimate average revenue per open store and further by location and region.

Most of other online data for Dunkin is from 2019 (before acquisition) and making a comparison between DNKN 2019 data and 2023 or even 2019 SBUX data is not relevant to their 2023 locations data (which likely has changed after the pandemic), which would make it an 'apples to oranges' comparison.

In [41]:
dd_2023_revenue = 1.4*1000000000
sb_2023_revenue = 36*1000000000

In [40]:
# number of stores by brand
sb_stores = merged_df['Starbucks Stores 2023'].sum()
dd_stores = merged_df['Dunkin Stores 2023'].sum()
# print(sb_stores, dd_stores)

In [42]:
# preliminary estimates based on revenue, before adding columns to merged_df
sb_rev_per_store = round(sb_2023_revenue/sb_stores)
dd_rev_per_store = round(dd_2023_revenue/dd_stores)
print(sb_rev_per_store, dd_rev_per_store)

2441671 149557
2441671 149557


considering a vast difference in revenues, while comparable difference in number of stores in the US, it makes it hard to believe that the above results on average revenue per location are valid. However, given there's no better data found, I proceed with the analysis for the sake of this assignment, regardless of the accuracy.

In [43]:
# adding new columns to dataset
merged_df['Starbucks est. Revenue per State in Million USD'] = round(merged_df['Starbucks Stores 2023']*sb_rev_per_store/1000000, 2)
merged_df['Dunkin est. Revenue per State in Million USD'] = round(merged_df['Dunkin Stores 2023']*dd_rev_per_store/1000000, 2)

In [44]:
merged_df.head()

,State,Starbucks Stores 2023,Dunkin Stores 2023,Population,Region,Starbucks est. Revenue per State in Million USD,Dunkin est. Revenue per State in Million USD
0,California,3080,134,39538223,West,7520.35,20.04
1,Texas,1346,196,30145505,South,3286.49,29.31
2,Florida,844,883,21538187,South,2060.77,132.06
3,Washington,741,0,7705281,West,1809.28,0.00
4,New York,692,1414,20201249,Northeast,1689.64,211.47


Now we can do some comparitive analysis with this data

## 3. Analysis

In [25]:
# What is the share of Strabucks and Dunkin stores per region?

# How does it compare to the populations in each region?

# How do revenues per region look like for each brand?

# What is the average revenue per capita in each state (in $1)?

In [45]:
merged_df.describe()

,Starbucks Stores 2023,Dunkin Stores 2023,Population,Starbucks est. Revenue per State in Million USD,Dunkin est. Revenue per State in Million USD
count,51.000000,51.000000,5.100000e+01,51.000000,51.000000
mean,289.098039,183.549020,6.518613e+06,705.882549,27.450392
std,477.755722,303.876377,7.470225e+06,1166.522916,45.447110
min,8.000000,0.000000,5.768510e+05,19.530000,0.000000
25%,56.500000,17.000000,1.816411e+06,137.955000,2.540000
50%,131.000000,50.000000,4.505836e+06,319.860000,7.480000
75%,332.000000,201.000000,7.428392e+06,810.630000,30.060000
max,3080.000000,1414.000000,3.953822e+07,7520.350000,211.470000


In [46]:
# merged_df.info()

In [47]:
# 1. Share of Strabucks and Dunkin stores per region # note: assisted by ChatGPT

region_store_counts = merged_df.groupby('Region')[['Starbucks Stores 2023', 'Dunkin Stores 2023']].sum()

# Calculate share of stores per region
region_store_counts['Starbucks Stores Share (%)'] = (region_store_counts['Starbucks Stores 2023'] /
                                             region_store_counts['Starbucks Stores 2023'].sum()) * 100
region_store_counts['Dunkin Stores Share (%)'] = (region_store_counts['Dunkin Stores 2023'] /
                                           region_store_counts['Dunkin Stores 2023'].sum()) * 100

# print(region_store_counts[['Starbucks Stores Share (%)', 'Dunkin Stores Share (%)']])

# note: I considered another option to do this analysis is using pd.crosstab, however it would require creating a dummy variable column eg. Starbucks = 1, Dunkin = 0. Given this approach would extend the number of rows, I opted not to do it. However, this would be possible with the dataset provided from automated function in part 4.

In [48]:
# also can join these results into a dataframe
stores_by_region = pd.DataFrame.join(region_store_counts['Starbucks Stores Share (%)'],region_store_counts['Dunkin Stores Share (%)'])
stores_by_region

,Starbucks Stores Share (%),Dunkin Stores Share (%)
Region,,
Midwest,16.739012,14.934302
Northeast,12.208356,53.872450
South,30.880358,27.518428
West,40.172273,3.674821


In [50]:
# How does the number of stores compare to the populations in each region? # note: assisted by ChatGPT
region_pop = merged_df.groupby('Region')['Population'].sum()
stores_to_pop = pd.concat([region_store_counts[['Starbucks Stores 2023', 'Dunkin Stores 2023']], region_pop], axis=1)

# Store count per capita (number of stores per 100,000 people for easier interpretation)
stores_to_pop['Starbucks Stores per 100k people'] = stores_to_pop['Starbucks Stores 2023'] / (stores_to_pop['Population']/100000)
stores_to_pop['Dunkin Stores per 100k people'] = stores_to_pop['Dunkin Stores 2023'] / (stores_to_pop['Population']/100000)

reg_stores_per_capita = pd.DataFrame.join(stores_to_pop['Starbucks Stores per 100k people'],stores_to_pop['Dunkin Stores per 100k people'])
reg_stores_per_capita

,Starbucks Stores per 100k people,Dunkin Stores per 100k people
Region,,
Midwest,3.577566,2.026514
Northeast,3.124504,8.753818
South,3.577543,2.024105
West,7.536719,0.437723


We can see that Dunkin has significantly more stores in Northeast compared to other regions, while Starbucks in the West. Which is not surprising given both brands origins. However, what was quite surprising is a relatively low presence of Dunkin stores in the West, also when compared to population.

In [51]:
# Revenue by region for each brand
region_revenue = merged_df.groupby('Region')[['Starbucks est. Revenue per State in Million USD', 
                                       'Dunkin est. Revenue per State in Million USD']].sum()

region_revenue

,Starbucks est. Revenue per State in Million USD,Dunkin est. Revenue per State in Million USD
Region,,
Midwest,6026.04,209.06
Northeast,4395.03,754.22
South,11116.91,385.25
West,14462.03,51.44


In [52]:
# Share of total revenue by region per brand # assisted by ChatGPT
total_revenue = region_revenue.sum()

# Calculate the share of total revenue per brand for each region
region_revenue['Starbucks Revenue Share (%)'] = (region_revenue['Starbucks est. Revenue per State in Million USD'] / total_revenue['Starbucks est. Revenue per State in Million USD']) * 100
region_revenue['Dunkin Revenue Share (%)'] = (region_revenue['Dunkin est. Revenue per State in Million USD'] / total_revenue['Dunkin est. Revenue per State in Million USD']) * 100

reg_rev_share = pd.DataFrame.join(round(region_revenue['Starbucks Revenue Share (%)'], 2),round(region_revenue['Dunkin Revenue Share (%)'], 2))
reg_rev_share

,Starbucks Revenue Share (%),Dunkin Revenue Share (%)
Region,,
Midwest,16.74,14.93
Northeast,12.21,53.87
South,30.88,27.52
West,40.17,3.67


In [53]:
# Revenue per capita for each state added to main dataframe merged_df # note: Assisted by ChatGPT
merged_df['Starbucks Revenue per Capita'] = (merged_df['Starbucks est. Revenue per State in Million USD'] * 1e6) / merged_df['Population']
merged_df['Dunkin Revenue per Capita'] = (merged_df['Dunkin est. Revenue per State in Million USD'] * 1e6) / merged_df['Population']

merged_df[['State', 'Starbucks Revenue per Capita', 'Dunkin Revenue per Capita']].head()

,State,Starbucks Revenue per Capita,Dunkin Revenue per Capita
0,California,190.204552,0.506851
1,Texas,109.020897,0.972284
2,Florida,95.679827,6.131435
3,Washington,234.810385,0.000000
4,New York,83.640373,10.468165


In [54]:
# correlations between number of stores and est. revenue per capita in each state # note: assisted by ChatGPT
starbucks_corr = merged_df[['Starbucks Revenue per Capita', 'Starbucks Stores 2023']].corr().iloc[0, 1]
dunkin_corr = merged_df[['Dunkin Revenue per Capita', 'Dunkin Stores 2023']].corr().iloc[0, 1]

print(starbucks_corr, dunkin_corr)

0.3765771295834709 0.5541588527688478
0.3765771295834709 0.5541588527688478


Interestingly, while both brands have more revenues in regions where their presence is highest, the correlation between revenue per capita and number of stores is higher for Dunkin than Starbucks. Considering a larger network of stores Starbucks and more presence in regions with larger populations, is a likely factors why this correlation metric is lower for Starbucks.

note: Moving forwards, once we learn running regressions in Python, it may be interesting to do more advanced modelling and analysis eg. for estimated revenue per capita in states or regions where competitor presence is large or lower, for example with linear models and interaction variables.

## 4. Automation

In [55]:
# given already created a function 'ws_to_df' to scrape the html data into a dataframe, the only adjustment required is to add a column with a brand name.

def ws_df_auto(url):
    # scraping the data with BeautifulSoup
    response = re.get(url)
    soup_table = BeautifulSoup(response.content, 'html.parser')

    # Parsing all rows from the table
    rows = soup_table.find_all('tr')

    # Extracting headers (if present)
    headers = [header.text.strip() for header in rows[0].find_all('th')]

    # Extract the brand name from the first header # assisted by ChatGPT
    brand_name = headers[1].split()[0] if len(headers) > 1 else "Unknown"

    # Initializing a list for the table
    table_data = []

    # Looping through rows to extract data
    for row in rows[1:]: # skipping header extracted in prior step
        state = row.find('a').text
        values = [state] + [td.text.strip().replace(',','') for td in row.find_all('td')] ## assisted by ChatGPT
        table_data.append(values)
    
    # Creating dataframe
    df = pd.DataFrame(table_data, columns=headers)
    
    # adding column with brand_name
    df['Brand'] = brand_name

    # Print result
    return(df)

In [56]:
ws_df_auto('https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state').head()

,State,Starbucks Stores 2023,Starbucks Stores 2021,Starbucks Stores 2024,Brand
0,California,3080,2959,3117,Starbucks
1,Texas,1346,1215,1409,Starbucks
2,Florida,844,786,892,Starbucks
3,Washington,741,739,736,Starbucks
4,New York,692,643,715,Starbucks


In [57]:
ws_df_auto('https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state').head()

,State,Dunkin Locations 2024,Dunkin Locations 2023,Brand
0,New York,1431,1414,Dunkin
1,Massachusetts,1042,1068,Dunkin
2,Florida,909,883,Dunkin
3,New Jersey,872,866,Dunkin
4,Illinois,711,692,Dunkin
